In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variable

open_api_key = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client

openai_client = OpenAI(api_key = open_api_key)


In [2]:
# ! pip install pdf2image

In [2]:
import importlib
import modules.pdf_extraction
importlib.reload(modules.pdf_extraction)
from modules.pdf_extraction import extract_text_from_pages, select_pdf_file


ModuleNotFoundError: No module named 'modules'

In [ ]:
pdf_path = select_pdf_file()

2025-02-08 23:03:07,739 - INFO - Opening file selection dialog.
2025-02-08 23:03:24,083 - INFO - Selected PDF file: C:/Users/derri/Desktop/page0027.pdf


In [ ]:
# 15
page_no = 0
extracted_text = extract_text_from_pages(pdf_path, pages=page_no)


In [ ]:
import pymupdf 
## 111
import base64
import requests

import importlib
import modules.llm
import modules.pdf_extraction
importlib.reload(modules.llm)
importlib.reload(modules.pdf_extraction)

from modules.llm import pattern_description_llm, table_extraction
from modules.pdf_extraction import get_page_pixel_data

base64_image = get_page_pixel_data(pdf_path=pdf_path, page_no=page_no, 
                    dpi = 300, image_type = 'png')

pattern_desc_from_image = pattern_description_llm(text_input = extracted_text, model='gpt-4o',  base64_image = base64_image, open_api_key=open_api_key)

pattern_desc_from_image

# Function to process each row in parallel
# open ai does not seem to have structed output for this image input version.
# Ill pass it as text to the next llm# 111

'1. Number of Tables on the Page: 7\n\n2. Table Headers: Metric Male 24° Cone Seat, S Series, DIN 3865 || Metric Female 24° Cone Seat, S Series, DIN 3865 || 90° Metric Female 24° Cone Seat, S Series, DIN 3865 || Standpipe, S Series || 45° Metric Female 24° Cone Seat, S Series, DIN 3865 || 45° Standpipe, S Series || 90° Standpipe, S Series\n\n3. Rules for Parsing Tables and Headers: \n   - Tables are often introduced with a descriptive header that includes metric specifications, usually in bold or distinct formatting.\n   - Each table is structured with clearly defined columns. Common column labels include "Thread," "OD mm," "To Suit Hose ID," and "Size."\n   - Headers may contain angle specifications (e.g., 24°, 45°, 90°) and component type descriptions (e.g., Cone Seat, Standpipe).\n   - Identify tables by looking for repeated column headers and consistent row patterns.\n   - Visual cues such as lines or spacing between lists can help determine the table boundaries.'

In [ ]:
1/0

ZeroDivisionError: division by zero

In [ ]:
import importlib
import modules.llm
importlib.reload(modules.llm)


from modules.llm import table_extraction


table_output = table_extraction(pattern_data = pattern_desc_from_image, text_data =extracted_text, model = "gpt-4o", openai_client=openai_client)
table_output

2025-02-08 21:18:43,831 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


['Category in WGIII',
 'Category description',
 'GHG emissions scenarios (SSPx-y*) in WGI & WGII',
 'RCPy** in WGI & WGII']

In [ ]:
pattern_desc_from_image

'The image and textual data indicate the presence of several organized tables. Here’s an analysis:\n\n### Table Structure and Organization:\n\n1. **Hydraulic Hose Range (Header Section):**\n   - **Title:** "HYDRAULIC HOSE RANGE"\n   - **Subtitle:** EN853-1, DIN20022 (Exceeds SAE 100R1AT)\n\n2. **Tables (Data Section):**\n   - **Table 1:** 1 Wire, 100 Metre Coils\n     - **Columns:** ID, OD mm, Max Pressure Bar, Bend Radius mm, Burst Pressure Bar\n     - **Rows:** Multiple entries detailing specifications for different hose IDs like 100R1AT-03-100, 100R1AT-04-100, etc.\n     - **Repeating Header Info:** Tech Sheet 5001\n\n   - **Table 2:** 1 Wire, 40 Metre Coils\n     - **Columns:** ID, OD mm, Max Pressure Bar, Bend Radius mm, Burst Pressure Bar\n     - **Rows:** Separate entries such as 100R1AT-20-40 and 100R1AT-24-40.\n\n   - **Table 3:** Technical Data - Hydraulic Hose, EN853, DIN20022, SAE100R1AT\n     - **Columns:** Part Number, Size mm/inch, Working Pressure bar/psi, Burst Pressur

In [ ]:
from pydantic import create_model
import json
import pandas as pd 

import importlib
import modules.llm
importlib.reload(modules.llm)
from modules.llm import llm_parser


import importlib
import modules.llm
importlib.reload(modules.llm)
from modules.llm import variable_extractor_llm

# Could targeting tables then extracting from each one by one and then concating be the best way to go ?
# how do i identify tables

# If user provided then

text_1 = """ 

Extract all data from 


"""
#  pattern = pattern_output,
# If no user provided then get a default template

pattern_output = pattern_description_llm(text_input=extracted_text, base64_image = base64_image, open_api_key= open_api_key )


var_list =  variable_extractor_llm(user_text=text_1, text_data=extracted_text, openai_client=openai_client, model='o3-mini') # gpt-4o


# Assume these are the field names coming from the front end
input_fields = list(set(var_list))

input_fields.append('table_number')  # To differential tables
print(input_fields)

#   Build a dictionary defining the field names and their types.
#  I will assume each field required a list of strings.
field_definitions = {field: (list[str], ...) for field in input_fields}

# Dynamically create a new Pydantic model 
dynamic_structured_output_class = create_model('structured_output', **field_definitions)


completion = llm_parser(user_text = text_1, 
                        target_variables= var_list,
                        openai_client = openai_client,
                        pattern=pattern_output,
                        text_document=extracted_text,
                        response_format=dynamic_structured_output_class,
                        model='o3-mini') # o3-mini gpt-4o

out  = completion.choices[0].message.content

data = json.loads(out)

# To ensure all columns are of the same length
# Determine the maximum length among the arrays
max_len = max(len(lst) for lst in data.values())

# Pad each list to have the same length
for key, lst in data.items():
    if len(lst) < max_len:
        lst.extend([None] * (max_len - len(lst)))

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

df # ensure columsn returnd are ascii clean white spaces

#error handling try and except 
# Perhaps retry when JSON error happens
# Factorise table numbers. Sometimes i can choose 2 with no other tables present

2025-02-06 23:44:40,496 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


['od_mm', 'to_suit_hose_id', 'size', 'header', '0309', 'thread', 'table_number']


2025-02-06 23:46:02,206 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


,od_mm,to_suit_hose_id,size,header,0309,thread,table_number
0,,3/16”,8S,"Metric Male 24¡ Cone Seat, S Series, DIN 3865",HMMS03-M16CF,M16 x 1.5,1
1,,1/4”,6S,"Metric Male 24¡ Cone Seat, S Series, DIN 3865",HMMS04-M14CF,M14 x 1.5,1
2,,1/4”,8S,"Metric Male 24¡ Cone Seat, S Series, DIN 3865",HMMS04-M16CF,M16 x 1.5,1
3,,1/4”,10S,"Metric Male 24¡ Cone Seat, S Series, DIN 3865",HMMS04-M18CF,M18 x 1.5,1
4,,5/16”,10S,"Metric Male 24¡ Cone Seat, S Series, DIN 3865",HMMS05-M18CF,M18 x 1.5,1
...,...,...,...,...,...,...,...
104,None,1”,,None,HSSP16-45-25CF,,7
105,None,1”,,None,HSSP16-45-30CF,,7
106,None,1”,,None,HSSP16-45-38CF,,7
107,None,1.1/4”,,None,HSSP20-45-30CF,,7


In [ ]:
1/0

In [ ]:
!pip install pdfplumber

  Using cached pdfplumber-0.11.5-py3-none-any.whl.metadata (42 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
  Using cached cryptography-44.0.0-cp39-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached pdfplumber-0.11.5-py3-none-any.whl (59 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
Using cached cryptography-44.0.0-cp39-abi3-win_amd64.whl (3.2 MB)
Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl (181 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)


In [ ]:
import pdfplumber
import pandas as pd

def extract_tables_from_pdf(pdf_path, output_csv=None):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted_tables = page.extract_tables()
            for table in extracted_tables:
                tables.append(pd.DataFrame(table))
    
    # Combine tables into one DataFrame if needed
    if tables:
        df = pd.concat(tables, ignore_index=True)
        if output_csv:
            df.to_csv(output_csv, index=False)
        return df
    else:
        return None

# Example usage

output_csv = "output.csv"  # Optional: specify a CSV file to save the extracted table
df = extract_tables_from_pdf(pdf_path=pdf_path, output_csv=output_csv)

if df is not None:
    print(df.head())  # Display first few rows
else:
    print("No tables found in the PDF.")


             0     1      2                   3                 4   \
0          0303    ID  OD mm  Max\nPressure\nBar  Bend\nRadius\nmm   
1          0303    ID  OD mm  Max\nPressure\nBar  Bend\nRadius\nmm   
2  Part\nNumber  Size   None   Working\nPressure              None   
3          None    mm   inch                 bar               psi   
4    100R1AT-03   4.7   3/16                 250              3625   

                     5      6            7     8                   9   \
0  Burst\nPressure\nBar    NaN          NaN   NaN                 NaN   
1  Burst\nPressure\nBar    NaN          NaN   NaN                 NaN   
2       Burst\nPressure   None  Bend Radius  None  Swaging Dimensions   
3                   bar    psi         Inch    mm   FT3 Type\nFerrule   
4                  1000  14503         3.54    90         FT3-12-03CF   

                  10                      11                          12  \
0                NaN                     NaN                    

In [ ]:
from img2table.document import PDF

doc = PDF(src=pdf_path, 
          pages=None,
          detect_rotation=False,
          pdf_text_extraction=True)

In [ ]:
from img2table.ocr import TesseractOCR
from img2table.document import Image
import pytesseract

# Instantiation of OCR
ocr = TesseractOCR(n_threads=1, lang="eng")

# Instantiation of document, either an image or a PDF
# doc = Image(pdf_path)

# Table extraction
extracted_tables = doc.extract_tables(ocr=ocr,
                                      implicit_rows=False,
                                      implicit_columns=False,
                                      borderless_tables=False,
                                      min_confidence=50)

OSError: Tesseract not found in environment. Check variables and PATH

In [ ]:
df.drop_duplicates(subset='id_information')

,id_information,table_number
0,HMMS03-M16CF,1
1,HMMS04-M14CF,1
2,HMMS04-M16CF,1
3,HMMS04-M18CF,1
4,HMMS05-M18CF,1
...,...,...
104,HSSP16-45-25CF,7
105,HSSP16-45-30CF,7
106,HSSP16-45-38CF,7
107,HSSP20-45-30CF,None


In [ ]:
extracted_text

'\n\n--- Page 1 ---\n\n6\n1.01    Hydraulic Hose & Inserts\nHydraulics\n1\n EN853-1, DIN20022 (Exceeds SAE 100R1AT)\n1 Wire, 40 Metre Coils\n0303\nID\nOD mm\nMax\nPressure\nBar\nBend\nRadius\nmm\nBurst\nPressure\nBar\n100R1AT-20-40\n1.1/4”\n44.8\n63\n420\n250\n100R1AT-24-40\n1.1/2”\n52\n50\n500\n200\n100R1AT-32-40\n2”\n65.9\n40\n630\n160\nTech Sheet 5001\n1 Wire, 100 Metre Coils\n0303\nID\nOD mm\nMax\nPressure\nBar\nBend\nRadius\nmm\nBurst\nPressure\nBar\n100R1AT-03-100\n3/16”\n12.5\n250\n90\n1000\n100R1AT-04-100\n1/4”\n14.1\n225\n100\n900\n100R1AT-05-100\n5/16”\n15.7\n215\n115\n850\n100R1AT-06-100\n3/8”\n18.1\n180\n130\n720\n100R1AT-08-100\n1/2”\n21.4\n160\n180\n640\n100R1AT-10-100\n5/8”\n24.5\n130\n200\n520\n100R1AT-12-100\n3/4”\n28.5\n105\n240\n420\n100R1AT-16-100\n1”\n36.6\n88\n300\n350\nTech Sheet 5001\nApplication:\nMedium pressure hydraulic applications\nSpecim cation:\nDIN 20022, EN853 – 1 Exceeds 100 R1AT\nConstruction:\n Inner tube: oil resistant synthetic rubber (NBR)\nReinf

In [ ]:
dynamic_structured_output_class

__main__.structured_output

In [ ]:
pdf_path

'C:/Users/derri/Desktop/page0012.pdf'

In [ ]:
print(dynamic_structured_output_class.__annotations__)


{'header': list[str], '0303': list[str], 'id': list[str], 'od_mm': list[str]}


In [ ]:
data

{'header': ['HYDRAULIC HOSE RANGE',
  'HYDRAULIC HOSE RANGE',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'id': ['0303',
  '0303',
  '0303',
  '0303',
  '0303',
  '0303',
  '0303',
  '0303',
  '0303',
  '0303',
  '0303',
  '0303',
  '0303',
  '0303',
  '0303',
  '0303'],
 'od_mm': ['44.8',
  '52',
  '65.9',
  '12.5',
  '14.1',
  '15.7',
  '18.1',
  '21.4',
  '24.5',
  '28.5',
  '36.6',
  '31.5',
  '38',
  '51',
  '25',
  '31.5']}

In [ ]:
# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

df

,order_number,model_number,heading
0,1051042861,NLESTOP37ENC,PUSH BUTTON STATIONS
1,1051042862,NLESTOP60ENC,PUSH BUTTON STATION ENCLOSURES
2,1051042863,NLESTOPKRENC,None
3,1051042864,NL2POSKSENC,None
4,1051042866,NLSSENC,None
5,1051042865,NLSENC,None
6,1051042878,NLPBENC1,None
7,1051042879,NLPBENC2,None


In [ ]:
extracted_text_list

['\n\n--- Page 5 ---\n\n\tPUSH BUTTON STATIONS\nOrder No.\nModel No.\nDescription\nContact Included\nPrice\nPUSH BUTTON STATIONS\n1051042861\nNLESTOP37ENC\nEmergency Stop Twist Release 37mm\n1 x NC\n£0.00\n1051042862\nNLESTOP60ENC\nEmergency Stop Twist Release 60mm\n1 x NC\n£0.00\n1051042863\nNLESTOPKRENC\nEmergency Stop Key Release 37mm\n1 x NC\n£0.00\n1051042864\nNL2POSKSENC\n2 Position Key Switch\n1 x NO\n£0.00\n1051042866\nNLSSENC\nStart/Stop\n1 x NO/1 x NC\n£0.00\n1051042865\nNLSENC\nStart\n1 x NO\n£0.00\n•\tManufactured in accordance with EN60947-5-1, \nEN60947-5-5 where applicable \n•\tIP65 rated (when correctly installed) \n•\tM20/PG16 gland entries\n•\t22mm push button range made for tough environments \n•\tInsulated self extinguishing ABS plastic housing\n•\tCommon auxiliaries throughout push button range\n•\tSpace for an additional contact block on each actuator\n•\tLight grey lid / dark grey base (yellow lid for Estop)\n•\tStainless steel captive screws\n•\tSupplied with 1 

In [ ]:
out  = completion.choices[0].message.content
out

'{"order_number":["1051042861","1051042862","1051042863","1051042864","1051042866","1051042865","1051042878","1051042879"],"model_number":["NLESTOP37ENC","NLESTOP60ENC","NLESTOPKRENC","NL2POSKSENC","NLSSENC","NLSENC","NLPBENC1","NLPBENC2"]}'

In [ ]:
# Parse the JSON string into a dictionary
import json
import pandas as pd 

# Display the DataFrame
print(df)

  order_number  model_number
0   1051042861  NLESTOP37ENC
1   1051042862  NLESTOP60ENC
2   1051042863  NLESTOPKRENC
3   1051042864   NL2POSKSENC
4   1051042866       NLSSENC
5   1051042865        NLSENC
6   1051042878      NLPBENC1
7   1051042879      NLPBENC2


In [ ]:
prompt_variable_extractor = """ 

 You are a skilled data extraction and parsing specialist with expertise in analysing text documents and extracting structured information, these could be from a varity of sources including PDFs and many others. 
          Your job is to extract data for the provided variable(s) from a text document in line with the user's request and return them in JSON format where each value is associated with each other.  

 
          ''''''''''''''''
          User's request: " I would like to extract the Order No and Model from the pdf document. The order number looks something like this (1051042839) and the model No is an mixture of numbers and letters and looks like this for instance NLOLC1
          I would also like to extract the heading which is usually uppercased and above the order and model number. 
          ''''''''''''''''
           
          Target variable(s): [model_number, order_number]
           
text document: ['\n\n--- Page 3 ---\n\n\tSTEEL ENCLOSED CONTACTORS\nOrder No.\nModel No.\nAC1 Heating & \nGeneral Load\nFluorescent \nMercury\nHalogen\nLED\nNo. of Poles\nDimensions HxWxD\nSTEEL ENCLOSED CONTACTORS\n1051042857\nNLCONM25/4\n25A\n16A\n13A\n25A\n4 + N\n184 x 184 x 149mm\n1051042858\nNLCONM45/4\n40A\n27A\n22A\n40A\n4 + N\n184 x 184 x 149mm\n1051042859\nNLCONM63/3N\n60A\n40A\n32A\n60A\n3 + N\n305 x 285 x 159mm\n•\tManufactured in accordance with EN60947-4-1 \n•\tIP55 steel enclosure\n•\tStandard coil voltage: 230V 50/60Hz\n•\tAuxiliary contacts can be fitted\n•\tColour RAL7035 light grey\n•\tKnockouts top and bottom\n•\tAll units are suitable for two wire control i.e. time \nswitch, thermostat, sensor or other remote controls\n•\tSupplied complete with neutral terminals\nNLCONM100/3N\n\tDIRECT ON LINE, REVERSING & \nSTAR DELTA STARTERS\n•\tManufactured in accordance with EN60947-4-1 \n•\tIP55 steel enclosure\n•\tColour RAL7035 light grey\n•\tSurface mounting\n•\tStandard coil voltages: 230V or 400V 50/60Hz\n•\tBuilt in start/stop and reset push button\n•\tHand reset thermal overloads\n•\tOverload to be ordered separately\n•\tStainless steel pozidrive fixing screws for lid\n•\tKnockouts top and bottom\n•\tInternal components DIN rail mounted\nOrder No.\nModel No.\nMax Rating\nControl Voltage\nDimensions H x W x D\nType\nDIRECT ON LINE & REVERSING STARTERS\n1051042849\nNLSTM27C\n7.5kW\n230V\n174 x 104 x 134mm\nDirect on Line\n£0.00\n1051042850\nNLSTM47C\n7.5kW\n400V\n174 x 104 x 134mm\nDirect on Line\n£0.00\n1051042856\nNLRVM45C\n5.5kW\n400V\n184 x 184 x 149mm\nReversing\n£0.00\n1051042851\nNLSTM25CSW\n5.5kW\n230V\n184 x 184 x 149mm\nDOL With Isolator\n£0.00\n1051042852\nNLSTM45CSW\n5.5kW\n400V\n184 x 184 x 149mm\nDOL With Isolator\n£0.00\n1051042854\nNLSDM415C\n15kW\n400V\n184 x 284 x 149mm\nStar Delta\n£0.00\nNLSTM27C\nStock items delivered FREE anywhere in UK\n15 3\n\n',
 '\n']

Output:

[
  {
    "heading": "STEEL ENCLOSED CONTACTORS",
    "order_number": "1051042857",
    "model_number": "NLCONM25/4"
  },
  {
    "heading": "STEEL ENCLOSED CONTACTORS",
    "order_number": "1051042858",
    "model_number": "NLCONM45/4"
  },
  {
    "heading": "STEEL ENCLOSED CONTACTORS",
    "order_number": "1051042859",
    "model_number": "NLCONM63/3N"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042849",
    "model_number": "NLSTM27C"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042850",
    "model_number": "NLSTM47C"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042856",
    "model_number": "NLRVM45C"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042851",
    "model_number": "NLSTM25CSW"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042852",
    "model_number": "NLSTM45CSW"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042854",
    "model_number": "NLSDM415C"
  }
]


"""


{'driver_name': list[str], 'stock_quantity': list[str], 'colour': list[str]}
